# Path Optimization (KOMO)



Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

**ry-c++-log** /home/jung-su/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



Let's add some objects

In [2]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["endeffWorkspace", obj1], ry.OT.sos, [1e0]);
komo.addObjective([1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

komo.optimize()
komo.getReport()

{'F_qItself/2-#46': {'order': 2.0, 'type': 'sos', 'sos': 67.4863494950677},
 'F_AccumulatedCollisions/0-#181': {'order': 0.0,
  'type': 'eq',
  'eq': 4.682643072273507e-05},
 'F_qLimits2/0-#181': {'order': 0.0,
  'type': 'ineq',
  'ineq': 2.858901965179978},
 'F_PairCollision/0-pr2R-item1': {'order': 0.0,
  'type': 'eq',
  'eq': 0.30839684430213654},
 'F_PositionDiff/0-endeffWorkspace-item1': {'order': 0.0,
  'type': 'sos',
  'sos': 2.0741288010095853},
 'F_qItself/1-#58': {'order': 1.0, 'type': 'eq', 'eq': 0.28631380102743176},
 'sos': 69.56047829607728,
 'ineq': 2.858901965179978,
 'eq': 0.594757471760291,
 'f': 0.0}

In [4]:
V = komo.view_play(False, 0.1)

Best way to read out: grab a configuration into K and analyze it here:

In [6]:
C.setFrameState( komo.getFrameState(19) )
C.getJointState()

array([-1.72082913e-01,  1.15146740e+00,  1.07242392e+00,  1.00014347e-01,
        2.35791817e-03, -2.32395474e-03,  6.54126314e-01,  1.20955890e+00,
        3.93041931e-01,  4.39971663e-03,  4.21079936e-01, -1.55024146e+00,
        9.15245624e-01, -7.23326760e-01, -2.07104007e+00, -3.31362206e-05,
        1.07580577e-05, -9.72506921e-02, -4.93388477e-01, -7.94239839e-05,
        0.00000000e+00,  9.76343569e-02,  9.65202910e-02,  1.19854247e-02,
        9.84224097e-03])